In [2]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import cv2 
import math 

**Functions definition**

In [10]:
def rgb_to_grayscale(im):
    """Converts a 3-channel RGB image to a single-channel grayscale image."""
    if len(im.shape) == 3:  # Only convert if the image is RGB
        return cv2.cvtColor(im, cv2.COLOR_RGB2GRAY)
    return im  # Return the image as is if it's already grayscale

def preprocess_image(image):
    """
    Preprocesses the image for line detection.

    Args:
        image: A 2D grayscale image (numpy array).

    Returns:
        A preprocessed grayscale image.
    """
    # Apply Gaussian blur for noise reduction
    image = cv2.GaussianBlur(image, (5, 5), 0)
    return image

def hough_line_transform(image, threshold=30, rho_resolution=0.5, theta_resolution=0.5):
    """
    Detects lines in a grayscale image using Hough Line Transform.

    Args:
        image: A 2D grayscale image (numpy array).
        threshold: Minimum number of votes required to consider a line.
        rho_resolution: Resolution for the rho parameter.
        theta_resolution: Resolution for the theta parameter.

    Returns:
        lines: A list of tuples (rho, theta) representing the detected lines.
    """
    # Calculate the maximum rho based on the image diagonal
    height, width = image.shape
    img_diagonal = np.ceil(np.sqrt(height**2 + width**2))
    max_rho = int(np.ceil(img_diagonal / rho_resolution)) * rho_resolution
    
    # Define parameter space for rhos and thetas
    rhos = np.arange(-max_rho, max_rho + 1, rho_resolution)
    thetas = np.deg2rad(np.arange(-90, 90, theta_resolution))

    # Create an empty Hough Accumulator
    accumulator = np.zeros((len(rhos), len(thetas)), dtype=np.uint64)

    # Iterate through all edge points in the image
    y_idxs, x_idxs = np.nonzero(image)
    for i in range(len(x_idxs)):
        x = x_idxs[i]
        y = y_idxs[i]

        for j in range(len(thetas)):
            rho = x * np.cos(thetas[j]) + y * np.sin(thetas[j])
            rho_idx = int(np.round((rho + max_rho) / rho_resolution))
            accumulator[rho_idx, j] += 1

    
    
    # Apply the threshold
    accumulator[accumulator < threshold] = 0

    # Find indices of non-zero values in thresholded accumulator
    rho_indices, theta_indices = np.nonzero(accumulator)
    rhos_detected = rhos[rho_indices]
    thetas_detected = thetas[theta_indices]
    
    # Combine rho and theta values into a single list of line parameters
    lines = list(zip(rhos_detected, thetas_detected))
    
    return lines

def visualize_lines(image, lines):
    """
    Visualizes the detected lines on the original image.

    Args:
        image: The original grayscale image.
        lines: A list of tuples (rho, theta) representing the detected lines.
    """
    # Convert image to RGB if it's grayscale
    if len(image.shape) == 2:
        image = cv2.cvtColor(image, cv2.COLOR_GRAY2BGR)

    # Create an empty line image for visualization
    line_image = np.zeros_like(image)
    
    # Iterate through the detected lines and draw them
    if lines is not None:
        for rho, theta in lines:
            a = np.cos(theta)
            b = np.sin(theta)
            x0 = rho * a
            y0 = rho * b
            x1 = int(x0 + 1000 * (-b))
            y1 = int(y0 + 1000 * a)
            x2 = int(x0 - 1000 * (-b))
            y2 = int(y0 - 1000 * a)
            cv2.line(line_image, (x1, y1), (x2, y2), (0, 255,0 ), 5)  

    # Overlay the lines onto the original image
    overlay_image = cv2.addWeighted(image, 0.5, line_image, 0.3, 0)
    
    # Display the overlay image
    cv2.imshow('Original Image', image)
    cv2.imshow('Detected Lines', overlay_image)
    
    # Wait for a key press to close the window
    cv2.waitKey(0)
    cv2.destroyAllWindows()


**Image upload and conversion**

In [11]:
image_path = 'utils/lines.png'  # Replace with your image path
image = np.array(Image.open(image_path).convert('L'))

    # Convert the image to grayscale if necessary
grayscale_image = rgb_to_grayscale(image)

    # Preprocess the image
preprocessed_image = preprocess_image(grayscale_image)

    # Apply Canny edge detection
edges = cv2.Canny(preprocessed_image, 50, 150)

    # Detect lines using the Hough Line Transform
lines = hough_line_transform(edges, threshold=150, rho_resolution=2, theta_resolution=1)

    # Visualize the detected lines on the original image
visualize_lines(grayscale_image, lines)

TypeError: hough_line_transform() missing 1 required positional argument: 'image'